In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
df

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.loc[df['age']<=25,'age_categorical']=1
df.loc[(df['age']<=50) & (df['age']>25),'age_categorical']=2
df.loc[(df['age']<=75) & (df['age']>50),'age_categorical']=3
df.loc[(df['age']<=100) & (df['age']>75),'age_categorical']=4

In [ ]:
df.gender.unique()

In [ ]:
df[['age_categorical','gender','heart_disease','bmi']].groupby(['age_categorical','gender','heart_disease'],as_index=False).mean()

In [ ]:
df.loc[(pd.isnull(df['bmi']))&(df['age_categorical']<=1) & (df['heart_disease']>1),'bmi']=29
df.loc[(pd.isnull(df['bmi']))&(df['age_categorical']<=1) & (df['heart_disease']>0),'bmi']=23
df.loc[(pd.isnull(df['bmi']))&(df['age_categorical']<=2) & (df['gender']!='Other'),'bmi']=30
df.loc[(pd.isnull(df['bmi']))&(df['age_categorical']<=2) & (df['gender']=='Other'),'bmi']=22
df.loc[(pd.isnull(df['bmi']))&(df['age_categorical']<=3),'bmi']=31
df.loc[(pd.isnull(df['bmi']))&(df['age_categorical']<=4),'bmi']=28



In [ ]:
df.columns

In [ ]:
col = ['gender','hypertension','heart_disease','ever_married','work_type','Residence_type','smoking_status','age_categorical']

In [ ]:
for c in col:
   print(c)
   print(df[c].unique())

In [ ]:
for c in col:
    print(df[[c,'stroke']].groupby([c],as_index=False).mean().sort_values(by='stroke'))
    print("\n")

In [ ]:
sns.barplot(x=df['gender'],y=df['stroke'],hue=df['smoking_status'])
plt.legend()
plt.show()

In [ ]:
sns.barplot(x=df['gender'],y=df['stroke'],hue=df['heart_disease'])
plt.legend()
plt.show()

In [ ]:
sns.barplot(x=df['gender'],y=df['stroke'],hue=df['hypertension'])
plt.legend()
plt.show()

In [ ]:
for i in col:
    for j in col:
        if(i!=j):
            print(df[[i,j,'stroke']].groupby([i,j],as_index=False).mean().sort_values(by='stroke',ascending=False))
            print("\n\n")
            

In [ ]:
for i in col:
    print(df[[i,'stroke']].groupby([i],as_index=False).mean().sort_values(by='stroke',ascending=False))
    print("\n\n")

In [ ]:
col.remove('hypertension')
col.remove('age_categorical')
col.remove('heart_disease')

In [ ]:
col

In [ ]:
for i in col:
    df = pd.concat([pd.get_dummies(df[i],drop_first=True),df],axis=1)
    df.drop([i],inplace=True,axis=1)
df

In [ ]:
df.drop(['id','age'],inplace=True,axis=1)

In [ ]:
import sklearn
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
df['bmi'] = scaler.fit_transform(df[['bmi']])
df['avg_glucose_level'] = scaler.fit_transform(df[['avg_glucose_level']])
df

In [ ]:
corr = df.corr()
plt.figure(figsize=(20,20))
sns.heatmap(corr,annot=True,square=True,cmap="YlGnBu")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [ ]:
y =df['stroke']
df.drop(['stroke'],inplace=True,axis=1)
x = df

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [ ]:
result = {}
models={}
models['KNN']=KNeighborsClassifier(n_neighbors=10)
models['RFC']=RandomForestClassifier(n_estimators=50)


In [ ]:
 for key in models:
    kfold = StratifiedKFold(n_splits=5,shuffle=True)
    cv_results = cross_val_score(models[key], x_train, y_train, cv=kfold, scoring='accuracy')
    result[key]=cv_results.mean()

In [ ]:
print(result)
chosen_one = max(result,key=result.get)

In [ ]:
model=models[chosen_one].fit(x_train,y_train)
predictions = model.predict(x_test)
print("Selected Model : "+chosen_one+"\nModel Accuracy on Testing Data : "+str(model.score(x_test,y_test)))